In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# ============================================================
# Utilities
# ============================================================

Nx = Ny = 32


def gaussian_forcing_2d(X, Y, mux, muy, sigx, sigy, amp=1.0):
    return amp * np.exp(
        -((X - mux)**2 / (2*sigx**2) + (Y - muy)**2 / (2*sigy**2))
    )


def compute_2d_poisson(x, y, f, bc):
    Nx = len(x)
    Ny = len(y)

    if Nx < 3 or Ny < 3:
        raise ValueError(f"Invalid grid: Nx={Nx}, Ny={Ny}")

    hx = x[1] - x[0]
    hy = y[1] - y[0]

    Ix = np.eye(Nx-2)
    Iy = np.eye(Ny-2)

    Dx = 2*np.eye(Nx-2) - np.diag(np.ones(Nx-3),1) - np.diag(np.ones(Nx-3),-1)
    Dy = 2*np.eye(Ny-2) - np.diag(np.ones(Ny-3),1) - np.diag(np.ones(Ny-3),-1)

    L = np.kron(Iy, Dx)/hx**2 + np.kron(Dy, Ix)/hy**2

    b = f[1:-1,1:-1].copy()
    b[0,:]  += bc["left"][1:-1]/hx**2
    b[-1,:] += bc["right"][1:-1]/hx**2
    b[:,0]  += bc["bottom"][1:-1]/hy**2
    b[:,-1] += bc["top"][1:-1]/hy**2

    u_inner = np.linalg.solve(L, b.reshape(-1))

    u = np.zeros((Nx,Ny))
    u[1:-1,1:-1] = u_inner.reshape(Nx-2,Ny-2)
    u[0,:]  = bc["left"]
    u[-1,:] = bc["right"]
    u[:,0]  = bc["bottom"]
    u[:,-1] = bc["top"]

    return u



def boundary_loss(u, bc):
    return (
        ((u[:,0,:]  - bc["left"])**2).mean() +
        ((u[:,-1,:] - bc["right"])**2).mean() +
        ((u[:,:,0]  - bc["bottom"])**2).mean() +
        ((u[:,:,-1] - bc["top"])**2).mean()
    )


def bc_features(bc):
    return torch.stack([
        bc["left"].mean(dim=1),
        bc["right"].mean(dim=1),
        bc["bottom"].mean(dim=1),
        bc["top"].mean(dim=1),
    ], dim=1)

# ============================================================
# DeepONet
# ============================================================

class MLP(nn.Module):
    def __init__(self, in_dim, out_dim, hidden=(256,256)):
        super().__init__()
        layers = []
        d = in_dim
        for h in hidden:
            layers += [nn.Linear(d,h), nn.Tanh()]
            d = h
        layers.append(nn.Linear(d,out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self,x):
        return self.net(x)


class DeepONet2D(nn.Module):
    def __init__(self, branch_dim, p=256):
        super().__init__()
        self.branch = MLP(branch_dim,p)
        self.trunk  = MLP(2,p)
        self.bias   = nn.Parameter(torch.zeros(1))

    def forward(self, branch_input, coords):
        b = self.branch(branch_input)        # (B,p)
        t = self.trunk(coords)               # (N,p)
        return (b[:,None,:]*t[None,:,:]).sum(-1) + self.bias


# ============================================================
# Trainer
# ============================================================

class Trainer:
    def __init__(self, model, coords, f, u, bc,
                 lr=1e-3, res_w=1.0, bc_w=50.0, device="cpu"):
        self.model = model.to(device)
        self.coords = coords.to(device).requires_grad_(True)
        self.f = torch.tensor(f, dtype=torch.float32, device=device)
        self.u = torch.tensor(u, dtype=torch.float32, device=device)
        self.bc = {k: torch.tensor(v, dtype=torch.float32, device=device)
                   for k,v in bc.items()}
        self.opt = torch.optim.Adam(model.parameters(), lr=lr)
        self.res_w = res_w
        self.bc_w  = bc_w
        self.B = f.shape[0]
        self.device = device

    def laplacian(self, u):
        g = torch.autograd.grad(u, self.coords,
                                torch.ones_like(u), create_graph=True)[0]
        ux, uy = g[:,0], g[:,1]
        uxx = torch.autograd.grad(ux, self.coords,
                                  torch.ones_like(ux), create_graph=True)[0][:,0]
        uyy = torch.autograd.grad(uy, self.coords,
                                  torch.ones_like(uy), create_graph=True)[0][:,1]
        return uxx + uyy

    def train(self, epochs=200):
        for ep in range(1,epochs+1):
            self.opt.zero_grad()

            branch = torch.cat([self.f, bc_features(self.bc)], dim=1)
            up = self.model(branch, self.coords)
            data_loss = ((up - self.u)**2).mean()

            res = 0.0
            for i in range(self.B):
                r = -self.laplacian(up[i]) - self.f[i]
                res += (r**2).mean()
            res /= self.B

            bc_l = boundary_loss(up.view(self.B,Nx,Ny), self.bc)
            loss = data_loss + self.res_w*res + self.bc_w*bc_l

            loss.backward()
            self.opt.step()

            if ep % 20 == 0 or ep == 1:
                print(f"Ep {ep:4d} | "
                      f"L {loss.item():.3e} | "
                      f"D {data_loss.item():.2e} | "
                      f"R {res.item():.2e} | "
                      f"BC {bc_l.item():.2e}")

# ============================================================
# Main
# ============================================================

np.random.seed(0)
torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

Nx = Ny = 32
x = np.linspace(0,1,Nx)
y = np.linspace(0,1,Ny)
X,Y = np.meshgrid(x,y,indexing="ij")
coords = torch.tensor(np.c_[X.ravel(),Y.ravel()], dtype=torch.float32)

N = 20
f_all, u_all = [], []
bc_all = {"left":[], "right":[], "bottom":[], "top":[]}

for _ in range(N):
    f = gaussian_forcing_2d(X,Y,
        *np.random.uniform(0.2,0.8,2),
        *np.random.uniform(0.05,0.15,2))
    f /= np.max(np.abs(f))
    bc = {
        "left":   np.random.uniform(-1,1,Ny),
        "right":  np.random.uniform(-1,1,Ny),
        "bottom": np.random.uniform(-1,1,Nx),
        "top":    np.random.uniform(-1,1,Nx),
    }
    u = compute_2d_poisson(x,y,f,bc)
    f_all.append(f.ravel())
    u_all.append(u.ravel())
    for k in bc: bc_all[k].append(bc[k])

f_all = np.array(f_all)
u_all = np.array(u_all)

u_scale = np.max(np.abs(u_all))
u_all /= u_scale

model = DeepONet2D(branch_dim=Nx*Ny+4, p=256)
trainer = Trainer(model, coords, f_all, u_all, bc_all,
                  lr=1e-3, res_w=1.0, bc_w=50.0, device=device)
trainer.train(epochs=200)

# ============================================================
# Test (unseen BCs)
# ============================================================

f_test = gaussian_forcing_2d(X,Y,0.35,0.72,0.08,0.12)
f_test /= np.max(np.abs(f_test))
bc_test = {
    "left":   np.sin(2*np.pi*y),
    "right":  np.cos(2*np.pi*y),
    "bottom": np.sin(2*np.pi*x),
    "top":    np.cos(2*np.pi*x),
}

u_true = compute_2d_poisson(x,y,f_test,bc_test)

with torch.no_grad():
    f_flat = torch.tensor(f_test.ravel()[None,:],dtype=torch.float32,device=device)
    bc_feat = torch.tensor([[bc_test[k].mean() for k in bc_test]],
                           dtype=torch.float32,device=device)
    u_pred = model(torch.cat([f_flat,bc_feat],1), coords.to(device))
    u_pred = u_pred.cpu().numpy().reshape(Nx,Ny)*u_scale

plt.figure(figsize=(12,3))
plt.subplot(131); plt.title("FD"); plt.imshow(u_true,origin="lower"); plt.colorbar()
plt.subplot(132); plt.title("DeepONet"); plt.imshow(u_pred,origin="lower"); plt.colorbar()
plt.subplot(133); plt.title("Error"); plt.imshow(np.abs(u_pred-u_true),origin="lower"); plt.colorbar()
plt.tight_layout(); plt.show()


ModuleNotFoundError: No module named 'torch'